In [1]:
%run Sucrose.ipynb

TO ANYONE LOOKING AT THIS CODE, PLEASE DON'T RUN THE ALROGTIHMS AGAIN, IT TAKES AGES TO GET THE RESULTS

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

The article had 0.154 mol-surfactin/mol-sucrose after knock-in. 

Sandie starts doing stuff from here:

Compute gene knockout strategies using algorithms like OptKnock, OptGene and OptCouple

In [2]:
#I did pip install cameo in my own terminal
import cameo

In [3]:
from cameo.strain_design.heuristic.evolutionary_based import OptGene

In [4]:
from cameo import models

In [5]:
optgene = OptGene(model)

In [6]:
#Testing if I found the right biomass name

In [7]:
model.reactions.BIOMASS_BS_10


Reaction identifier,BIOMASS_BS_10
Name,B subtilis biomass demand
Memory address,0x01eb72296670
Stoichiometry,0.000367 10fthf_c + 0.266902 ala__L_c + 0.00467 amp_c + 0.193021 arg__L_c + 0.147987 asn__L_c + 0.148014 asp__L_c + 105.053483 atp_c + 0.003205 ca2_c + 5e-06 cdlp_BS_c + 0.000251 cdp_c + 0.001042... 0.000367 10-Formyltetrahydrofolate + 0.266902 L-Alanine + 0.00467 AMP C10H12N5O7P + 0.193021 L-Arginine + 0.147987 L-Asparagine + 0.148014 L-Aspartate + 105.053483 ATP C10H12N5O13P3 + 0.003205...
GPR,
Lower bound,0.0
Upper bound,999999.0


In [8]:
model.reactions.query('biomass','name')

[<Reaction BIOMASS_BS_10 at 0x1eb72296670>]

In [9]:
model.reactions.DM_surfactin_c

Reaction identifier,DM_surfactin_c
Name,demand
Memory address,0x01eb72af8970
Stoichiometry,surfactin_c --> -->
GPR,
Lower bound,0
Upper bound,1000.0


In [10]:
model.reactions.EX_sucr_e

Reaction identifier,EX_sucr_e
Name,Sucrose exchange
Memory address,0x01eb72138fa0
Stoichiometry,sucr_e <=> Sucrose C12H22O11 <=>
GPR,
Lower bound,-1000
Upper bound,999999.0


Will this ever work if we don't have the genes?

In [11]:
result_optgene = optgene.run(target="DM_surfactin_c",
                     biomass="BIOMASS_BS_10",
                     substrate="EX_sucr_e",
                     max_evaluations=20000, max_knockouts=3, plot=False)

Starting optimization at Tue, 16 Nov 2021 13:28:48


HBox()

Finished after 00:13:38


In [12]:
result_optgene

No solutions found


,reactions,genes,size,fva_min,fva_max,target_flux,biomass_flux,yield,fitness


In [13]:
from cameo.strain_design.deterministic.linear_programming import OptKnock

In [14]:
optknock = OptKnock(model, fraction_of_optimum=0.1)

C:\Users\Natasha\anaconda3\envs\Cell factory\lib\site-packages\cameo\strain_design\deterministic\linear_programming.py:118: UserWarning: You are trying to run OptKnock with glpk_interface. This might not end well.
  warnings.warn("You are trying to run OptKnock with %s. This might not end well." %


In [ ]:
result_optknock = optknock.run(max_knockouts=1, target="DM_surfactin_c", biomass="BIOMASS_BS_10")

In [ ]:
result_optknock

This seems susppicious because the there's no biomass production

It might be because surfactin is toxic and kills the ceLL

Or it might be because we have so many different substrates in order to produce our product, so if we knock out one thing it might adversely affect another. 

We'll try and manually remove the ADMDC reaction to see what happens

In [ ]:
model.reactions.PPM2

In [ ]:
model.reactions.PPM2.lower_bound = 0

In [39]:
model.reactions.PPM2.upper_bound = 0

In [40]:
model.reactions.PPM2

Reaction identifier,PPM2
Name,Phosphopentomutase 2 (deoxyribose)
Memory address,0x021ede810e20
Stoichiometry,2dr1p_c --> 2dr5p_c 2-Deoxy-D-ribose 1-phosphate --> 2-Deoxy-D-ribose 5-phosphate
GPR,BSU23500
Lower bound,0
Upper bound,0


In [41]:
solution = model.optimize()

In [42]:
solution

,fluxes,reduced_costs
EX_etha_e,0.000000,-1.428571e-01
EX_drib_e,0.000000,0.000000e+00
EX_csn_e,0.000000,-4.285714e-01
EX_5mtr_e,0.000000,-4.285714e-01
EX_dtmp_e,0.000000,-2.857143e-01
...,...,...
SP_1,0.357143,-2.220446e-16
SP_2,0.000000,-2.442491e-15
SP_3,0.000000,-5.329071e-15
SP_4,0.000000,2.359224e-15


In [43]:
model.reactions.PPM2.flux

0.0

In [34]:
model.objective = model.reactions.DM_surfactin_c
surfactin_production = solution.objective_value
print('Maximum productivity =', surfactin_production, 'mmol/gDW*h')
   
maximum_yield = surfactin_production / (-1*(model.reactions.EX_sucr_e.flux))
print('Maximum theoretical yield =', maximum_yield, 'mmol-surfactin/mmol-glucose')

Maximum productivity = 0.6241848146320885 mmol/gDW*h
Maximum theoretical yield = 0.21359453201850118 mmol-surfactin/mmol-glucose


In [35]:
print(model.objective) #Test

Maximize
1.0*DM_surfactin_c - 1.0*DM_surfactin_c_reverse_e4782


In [30]:
#I might need to double check that cell doesn't die because of this knock out

In [44]:
model.reactions.BIOMASS_BS_10

Reaction identifier,BIOMASS_BS_10
Name,B subtilis biomass demand
Memory address,0x021ede364c10
Stoichiometry,0.000367 10fthf_c + 0.266902 ala__L_c + 0.00467 amp_c + 0.193021 arg__L_c + 0.147987 asn__L_c + 0.148014 asp__L_c + 105.053483 atp_c + 0.003205 ca2_c + 5e-06 cdlp_BS_c + 0.000251 cdp_c + 0.001042... 0.000367 10-Formyltetrahydrofolate + 0.266902 L-Alanine + 0.00467 AMP C10H12N5O7P + 0.193021 L-Arginine + 0.147987 L-Asparagine + 0.148014 L-Aspartate + 105.053483 ATP C10H12N5O13P3 + 0.003205...
GPR,
Lower bound,0.0
Upper bound,999999.0


In [45]:
model.reactions.BIOMASS_BS_10.flux

0.0

In [46]:
#It seems like our cell dies??